In [114]:
from sqlvis import vis
import sqlite3

conn = sqlite3.connect('../../data/shopping.db')

In [115]:
# Retrieve the schema from the db connection
schema = vis.schema_from_conn(conn)
schema

# Missing: Shopping database diagram

## Demonstration: normal query

In [140]:
query = '''
SELECT p.price, c.cName
FROM customer AS c, purchase AS p
WHERE c.cID = p.cID
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

In [141]:
query = '''
SELECT p.date, SUM(p.price)
FROM purchase AS p
WHERE quantity < 5
GROUP BY p.date
HAVING COUNT(tID) > 50
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Demonstration: query with error

In [142]:
query = '''
SELECT SUM(p.price)
FROM purchase AS p
WHERE p.quantity > 10
GROUP BY p.date
HAVING COUNT(p.pID) < 5
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 1

Issues:
- cName is tagged in red, but the cell isn't highlighted or autoexpanded and there is no tooltip?
    - Update: I implemented selection object handling, and now it's not even tagged in red anymore... not better.

In [143]:
query = '''
SELECT MAX(p.price), MIN(p.price), cName
WHERE c.cID = p.cID
FROM customer AS c, purchase AS p
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 2

Issues here:
- Where in the SELECT the error occurs defines whether it is marked as a problem or not?
- In- or excluding the `p.` changes whether it is visualized (may be because vis doesn't know which table to attach the error to? But the highlight is still made... huh?)

In [150]:
# Commented versions are different error variations, for debug reasons
query = '''
SELECT MAX(p.price), date, MIN(p.price), cName
FROM customer AS c, purchase AS p
WHERE c.cID = p.cID
GROUP BY cName, c.city
'''

# query = '''
# SELECT MAX(p.price), MIN(p.price), cName, date
# FROM customer AS c, purchase AS p
# WHERE c.cID = p.cID
# GROUP BY cName, c.city
# '''

# query = '''
# SELECT p.date, MAX(p.price), MIN(p.price), cName
# FROM customer AS c, purchase AS p
# WHERE c.cID = p.cID
# GROUP BY cName, c.city
# '''

# query = '''
# SELECT MAX(p.price), MIN(p.price), cName, p.date
# FROM customer AS c, purchase AS p
# WHERE c.cID = p.cID
# GROUP BY cName, c.city
# '''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 3

Issues here:
- GROUP BY mistake not highlighting node, also no tooltip (this error will also disappear when the one below is fixed)
- WHERE should be late instead of GROUP BY early (@root level)
- Side error: referencing error in subquery is not detected? (try making it s.cit) May be related to me breaking selections though

In [152]:
query = '''
SELECT MAX(p.price), MIN(p.price), cName
FROM customer AS c, purchase AS p
GROUP BY cName
HAVING p.quantity > 5
WHERE c.cID = p.cID
AND c.city IN (FROM store AS s
               SELECT s.city)
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 4

Issues:
- Tooltip boxes are too small, that's the only issue really.

In [153]:
query = '''
SELECT s.sName, c.cName, SUM(p.quantity)
FROM customer AS c, purchase AS p, store AS s
WHERE c.cID = p.cID
WHERE s.city = c.city
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 5

Issues:
- The WHERE with aggregation triggers changing the entire WHERE to HAVING, which breaks more things.
- Late keyword detection is triggered incorrectly (I thought that wasn't even reachable?), why? And also, why does that crash this query?

In [154]:
query = '''
SELECT s.sName, c.cName, SUM(p.quantity)
FROM customer AS c, purchase AS p, store AS s
WHERE c.cID = p.cID
AND s.city = c.city
AND COUNT(p.pID) < 10
GROUP BY c.cName, s.sName
HAVING pr.pName LIKE '%a%'
AND s.sName IN (FROM store AS s2
                SELECT s2.sName
                WHERE s2.city = 'Eindhoven'
               )
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 6

Issues:
- How is the parsing crashing?????
- Late keyword detection is triggered, and once again crashes the system on insertErrorInfo. How?????

In [155]:
query = '''
WITH confusion AS (
    FROM store AS s1
    WHERE s1.sName LIKE '%e%'
    AND COUNT(s1.sID) > 2
    GROUP BY s1.city
    HAVING s1.city IN (FROM customer AS c1
                       GROUP BY c1.city
                       HAVING COUNT(c1.cID) > 20
                       SELECT c1.city
                      )
    SELECT s1.sName)

FROM confusion AS co, purchase AS pur, store as s2
WHERE s2.sName = co.sName
AND s2.sID = pur.sID
SELECT co.sName, pur.date
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 7

Issues:
- Double HAVING issue is not being shown at all.
- Because of it being double HAVING, the `cName LIKE '%a%'` is not visualized?
- The `cName LIKE '%a%'`, when visualized, is shown as a WHERE component (due to it not containing aggregation).

In [157]:
query = '''
SELECT MAX(p.price), MIN(p.price)
FROM customer AS c, purchase AS p
WHERE c.cID = p.cID
GROUP BY c.city
HAVING AVG(p.quantity) > 5
HAVING c.cName LIKE '%a%'
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 8

Issues: no issues, working as intended

In [158]:
query = '''
SELECT MAX(p.price), MIN(p.price), p.date
FROM purchase AS p
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 9

Issues:
- This query is a bit screwed up. Need to change the query itself, because this doesn't make sense. Check Onedrive spreadsheet for what should be in here.

In [162]:
query = '''
SELECT c.cName, MAX(p.price)
FROM customer AS c, purchase AS p
WHERE c.cName IN (GROUP BY p2.cName
                  SELECT p2.cName
                  FROM purchase AS p2
                  WHERE p2.cName LIKE '%a%'
                  WHERE COUNT(p2.cID) < 5
                  HAVING SUM(p2.price) > 20
                 )
AND c.cID = p.cID
GROUP BY c.cName
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

## Problem 10

Issues:
- p.date is made orange as if it is being selected... it is not. That is the entire problem for incorr grouping.
- (`c.`)`cName` is not being marked as insufficient grouping.
- s.sName is not being marked as insufficient grouping.

In [165]:
query = '''
SELECT MAX(p.price) as highest_purchase, MIN(p.price), cName, s.sName, street
FROM customer AS c, purchase AS p, store AS s
WHERE c.cID = p.cID
AND s.sID = p.sID
GROUP BY p.date
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)